# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [7]:
# Google Chrome -> Chromedriver
# Firefox -> Geckodriver

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#Chrome invisível
#from selenium.webdriver.chrome.option import Options
#    chrome_options = Options()
#    chorme_options.headless = True
#    navegador = webdriver.Chrome(options=chrome_options)

navegador = webdriver.Chrome()

# Passo 1: Pegar a cotação do dólar
# Entrar no Google
navegador.get("https://www.google.com/")

# xpath -> identificador de cada elemento do site
# descobrir xpath utilizando a ferramente "inspeciosar", com o botão direito do mouse
# utilizar aspas simples para xpath para não bugar com o caminho inserido
navegador.find_element('xpath',
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element('xpath',
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element('xpath',
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

# Passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com/")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

# Passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")

print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)

navegador.quit()

5.0633
5.504060265000001
323.48


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [8]:
# Passo 4: Importar a base de dados e Atualizar a base

import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [9]:
# Passo 5: Recalcular os preços

# atualizar a cotação
# .loc = localizar / parâmento (linha, coluna)

# cotação dólar
tabela.loc[tabela["Moeda"] == "Dólar","Cotação"] = float(cotacao_dolar)

# cotação euro
# testando outro modo de citar nome de coluna
tabela.loc[tabela.Moeda == "Euro","Cotação"] = float(cotacao_euro)

# cotação ouro
tabela.loc[tabela ["Moeda"] == "Ouro","Cotação"] = float(cotacao_ouro)

# recalcular os preços

# preço de compra = cotação * preço original
tabela["Preço de Compra"] = tabela["Cotação"] * tabela["Preço Original"]

# preço de venda = preço de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.06330,5063.249367,1.40,7088.549114
1,Carro Renault,4500.00,Euro,5.50406,24768.271193,2.00,49536.542385
2,Notebook Dell,899.99,Dólar,5.06330,4556.919367,1.70,7746.762924
3,IPhone,799.00,Dólar,5.06330,4045.576700,1.70,6877.480390
4,Carro Fiat,3000.00,Euro,5.50406,16512.180795,1.90,31373.143510
5,Celular Xiaomi,480.48,Dólar,5.06330,2432.814384,2.00,4865.628768
6,Joia 20g,20.00,Ouro,323.48000,6469.600000,1.15,7440.040000


### Agora vamos exportar a nova base de preços atualizada

In [10]:
# Passo 6: Exportar a base atualizada
tabela.to_excel("Produtos Novo.xlsx", index = False)

In [11]:
# instalar selenium
!pip install selenium